In [1]:
from packaging import version

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
import tensorboard as tb
import tensorflow as tf
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import os
import numpy as np

In [2]:
major_ver, minor_ver, _ = version.parse(tb.__version__).release
assert major_ver >= 2 and minor_ver >= 3, \
    "This notebook requires TensorBoard 2.3 or later."
print("TensorBoard version: ", tb.__version__)

TensorBoard version:  2.7.0


In [4]:
env_list = ['atari_seaquest', 'atari_enduro', 'atari_pong']
agent_list = ['agent_1', 'agent_2', 'agent_3', 'patch_size_1', 'patch_size_4', 'patch_size_16']
seed_list = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
metric_list = ['scalars/train_return', 'scalars/eval_resample_jitter_inter_avg_return', 'scalars/eval_resample_jitter_extra_avg_return', 'scalars/eval_rgb_avg_return']
dir_path = '../../../../mnt/storage/split_dreamer_logs/'
def extract_score():
    score_array = np.zeros((len(env_list), len(agent_list), len(metric_list), len(seed_list)))
    for env_idx, env in enumerate(env_list):
        for agent_idx, agent in enumerate(agent_list):
            for metric_idx, metric in enumerate(metric_list):
                for seed_idx, seed in enumerate(seed_list):

                    file_path = os.path.join(dir_path, env, agent, seed)
                    print(file_path)
                    # read from summary writer
                    event_acc = EventAccumulator(file_path, size_guidance={'tensors': 0})
                    event_acc.Reload()
                    w, s, t = event_acc.Tensors(metric)[-1]
                    score_array[env_idx, agent_idx, metric_idx, seed_idx] = tf.make_ndarray(t)
    return score_array
#                 dataframe = pd.DataFrame([(w, s, tf.make_ndarray(t)) for w, s, t in event_acc.Tensors('scalars/eval_resample_jitter_extra_avg_return')],
#                 columns=['wall_time', 'step', 'tensor'])
# Save
score_array = extract_score()
np.save(os.path.join(dir_path, "scores.npy"), score_array)
# Load
score_array = np.load(os.path.join(dir_path, "scores.npy"))

../../../../mnt/storage/split_dreamer_logs/atari_seaquest/agent_1/1
../../../../mnt/storage/split_dreamer_logs/atari_seaquest/agent_1/2
../../../../mnt/storage/split_dreamer_logs/atari_seaquest/agent_1/3
../../../../mnt/storage/split_dreamer_logs/atari_seaquest/agent_1/4
../../../../mnt/storage/split_dreamer_logs/atari_seaquest/agent_1/5
../../../../mnt/storage/split_dreamer_logs/atari_seaquest/agent_1/6
../../../../mnt/storage/split_dreamer_logs/atari_seaquest/agent_1/7
../../../../mnt/storage/split_dreamer_logs/atari_seaquest/agent_1/8
../../../../mnt/storage/split_dreamer_logs/atari_seaquest/agent_1/9
../../../../mnt/storage/split_dreamer_logs/atari_seaquest/agent_1/10
../../../../mnt/storage/split_dreamer_logs/atari_seaquest/agent_1/1
../../../../mnt/storage/split_dreamer_logs/atari_seaquest/agent_1/2
../../../../mnt/storage/split_dreamer_logs/atari_seaquest/agent_1/3
../../../../mnt/storage/split_dreamer_logs/atari_seaquest/agent_1/4
../../../../mnt/storage/split_dreamer_logs/atar

In [14]:
score_mean = np.int_(list(np.mean(score_array, axis = 3)))
score_std = np.int_(list(np.std(score_array, axis = 3)))

In [16]:
score_array.shape

(3, 6, 4, 10)

In [6]:
print(score_mean[1,:,:].shape)

(6, 4)


In [8]:
columns_name = ['inter_avg_return', 'extra_avg_return', 'rgb_avg_return']
rows_name = ['Baseline', 'Upper Bound (Trained with full range jittering)', 'SPLIT Patch Size: 8', 'SPLIT Patch Size: 1' , 'SPLIT Patch Size: 4', 'SPLIT Patch Size: 16']
seaquest_returns = pd.DataFrame(score_mean[0, :, 1:], 
                                 columns=columns_name, 
                                index=rows_name)
enduro_returns = pd.DataFrame(score_mean[1, :, 1:], 
                                 columns=columns_name, 
                                index=rows_name)
pong_returns = pd.DataFrame(score_mean[2, :, 1:], 
                                 columns=columns_name, 
                                index=rows_name)

seaquest_returns_std = pd.DataFrame(score_std[0, :, 1:], 
                                 columns=columns_name, 
                                index=rows_name)
enduro_returns_std = pd.DataFrame(score_std[1, :, 1:], 
                                 columns=columns_name, 
                                index=rows_name)
pong_returns_std = pd.DataFrame(score_std[2, :, 1:], 
                                 columns=columns_name, 
                                index=rows_name)



In [9]:
seaquest_returns.style.highlight_max(color='red',axis=0)
# Seaquest Returns

,inter_avg_return,extra_avg_return,rgb_avg_return
Baseline,4844,104,4719
Upper Bound (Trained with full range jittering),2374,2301,2387
SPLIT Patch Size: 8,5831,362,6122
SPLIT Patch Size: 1,9240,1578,9160
SPLIT Patch Size: 4,5662,319,5722
SPLIT Patch Size: 16,7689,1328,7795


In [10]:
seaquest_returns_std

,inter_avg_return,extra_avg_return,rgb_avg_return
Baseline,2142,59,2187
Upper Bound (Trained with full range jittering),1287,1382,1502
SPLIT Patch Size: 8,2379,162,2861
SPLIT Patch Size: 1,7718,544,7349
SPLIT Patch Size: 4,2622,164,2845
SPLIT Patch Size: 16,1567,642,1153


In [11]:
enduro_returns.style.highlight_max(color='red',axis=0)
# Enduro Returns

,inter_avg_return,extra_avg_return,rgb_avg_return
Baseline,1431,183,1443
Upper Bound (Trained with full range jittering),964,830,1053
SPLIT Patch Size: 8,1671,399,1725
SPLIT Patch Size: 1,1802,530,1924
SPLIT Patch Size: 4,1648,401,1739
SPLIT Patch Size: 16,2020,688,2089


In [12]:
enduro_returns_std

,inter_avg_return,extra_avg_return,rgb_avg_return
Baseline,271,59,302
Upper Bound (Trained with full range jittering),215,196,206
SPLIT Patch Size: 8,276,94,243
SPLIT Patch Size: 1,124,163,149
SPLIT Patch Size: 4,215,100,274
SPLIT Patch Size: 16,167,174,129


In [13]:
pong_returns.style.highlight_max(color='red',axis=0)
# Pong Returns

,inter_avg_return,extra_avg_return,rgb_avg_return
Baseline,18,-14,18
Upper Bound (Trained with full range jittering),15,15,15
SPLIT Patch Size: 8,18,6,18
SPLIT Patch Size: 1,17,-3,16
SPLIT Patch Size: 4,15,6,15
SPLIT Patch Size: 16,18,15,18


In [17]:
pong_returns_std

,inter_avg_return,extra_avg_return,rgb_avg_return
Baseline,1,4,0
Upper Bound (Trained with full range jittering),1,1,1
SPLIT Patch Size: 8,0,6,0
SPLIT Patch Size: 1,1,11,1
SPLIT Patch Size: 4,6,8,6
SPLIT Patch Size: 16,1,2,1
